In [1]:
import os
import torch
import torch.nn.functional as F
from tqdm import tqdm
from torch_geometric.loader import NeighborLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.nn import MessagePassing, SAGEConv
from ogb.nodeproppred import Evaluator, PygNodePropPredDataset
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch_geometric.data import Data
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt
from early_stopping import EarlyStopping
import random
from yf_dataset import getInput

seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
train_idx, eval_idx, test_idx, weight, yfdata, gnnInputData = getInput(False, False)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.load(r'C:\Users\tony\Desktop\GNN_test\result\CrossEntrophyLoss\CrossEntrophyLoss startLr=1.813187e-04 numNeighbors=14\best_model.pt')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


c:\Users\tony\Desktop\GNN_test\yf_dataset.py:73: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if np.isnan(data[price_col][i]):
c:\Users\tony\Desktop\GNN_test\yf_dataset.py:76: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  local_df.append(data[price_col][i])


If there is NAN in data?  False
Index(['2011-01-31 00:00:00 ~ 2011-02-09 00:00:00',
       '2011-02-01 00:00:00 ~ 2011-02-10 00:00:00',
       '2011-02-02 00:00:00 ~ 2011-02-11 00:00:00',
       '2011-02-03 00:00:00 ~ 2011-02-14 00:00:00',
       '2011-02-04 00:00:00 ~ 2011-02-15 00:00:00',
       '2011-02-07 00:00:00 ~ 2011-02-16 00:00:00',
       '2011-02-08 00:00:00 ~ 2011-02-17 00:00:00',
       '2011-02-09 00:00:00 ~ 2011-02-18 00:00:00',
       '2011-02-10 00:00:00 ~ 2011-02-22 00:00:00',
       '2011-02-11 00:00:00 ~ 2011-02-23 00:00:00',
       ...
       '2019-08-08 00:00:00 ~ 2019-08-19 00:00:00',
       '2019-08-09 00:00:00 ~ 2019-08-20 00:00:00',
       '2019-08-12 00:00:00 ~ 2019-08-21 00:00:00',
       '2019-08-13 00:00:00 ~ 2019-08-22 00:00:00',
       '2019-08-14 00:00:00 ~ 2019-08-23 00:00:00',
       '2019-08-15 00:00:00 ~ 2019-08-26 00:00:00',
       '2019-08-16 00:00:00 ~ 2019-08-27 00:00:00',
       '2019-08-19 00:00:00 ~ 2019-08-28 00:00:00',
       '2019-08-20 00

c:\Users\tony\Desktop\GNN_test\yf_dataset.py:114: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  edge_index = torch.tensor([


feature_matrix_shape =  torch.Size([2154, 35])
edge size=  torch.Size([2, 34423])
y =  torch.Size([2154, 1])
找到日期 '2013-01-18 00:00:00 ~ 2013-01-30 00:00:00'，index= 495
找到日期 '2017-05-12 00:00:00 ~ 2017-05-23 00:00:00'，index= 1581
找到日期 '2018-03-16 00:00:00 ~ 2018-03-27 00:00:00'，index= 1793
找到日期 '2019-08-20 00:00:00 ~ 2019-08-29 00:00:00'，index= 2152
上漲(0)的数量： 534
持有(1)的数量： 1203
下跌(2)的数量： 417
weight:  [4.033707865093001, 1.7905236907581836, 5.165467625775408]


In [3]:
total_loader = NeighborLoader(gnnInputData, input_nodes=test_idx, num_neighbors=[-1],
                                shuffle=False,
                                num_workers=os.cpu_count() - 2)

In [4]:
model.eval()
model.to(device)
out, var = model.inference(total_loader, device)
y_pred = out.argmax(dim=-1) 

In [16]:
label_data = yfdata['2018-03-28':]
label_data = label_data[:-1]
init_investment = 10000
investment = init_investment
sAndp_hold = 0
total = []
sell_count = 0
buy_count = 0
for index, pred in enumerate(y_pred):
    pred = pred.numpy()
    open = label_data.iloc[index]['Open']
    adj_close = label_data.iloc[index]['Adj Close']

    if ((pred == 0) & (investment >= adj_close)):
        investment -= adj_close
        sAndp_hold += 1
        buy_count += 1
    elif ((pred == 2) & (sAndp_hold > 0)):
        investment += adj_close
        sAndp_hold -= 1
        sell_count += 1

    total.append(investment + (sAndp_hold * adj_close))
          
total

[10000.0,
 10000.0,
 10000.0,
 10000.0,
 10030.239990234375,
 10048.39013671875,
 9990.02001953125,
 9998.7099609375,
 10086.13037109375,
 10042.08984375,
 10107.489990234375,
 10084.420166015625,
 10149.040283203125,
 10234.689697265625,
 10241.439697265625,
 10194.90966796875,
 10125.939697265625,
 10126.39013671875,
 10019.2001953125,
 10033.7197265625,
 10116.33984375,
 10125.249755859375,
 10059.670166015625,
 10079.920166015625,
 10022.52978515625,
 10004.7099609375,
 10105.77978515625,
 10133.40966796875,
 10131.27978515625,
 10208.89013671875,
 10284.730224609375,
 10298.679931640625,
 10305.90966796875,
 10249.869873046875,
 10282.89990234375,
 10275.90966796875,
 10254.429931640625,
 10314.550048828125,
 10288.83984375,
 10315.39013671875,
 10298.800048828125,
 10279.51025390625,
 10185.100341796875,
 10287.550048828125,
 10231.330078125,
 10290.0302734375,
 10326.7802734375,
 10332.570068359375,
 10403.22021484375,
 10397.2802734375,
 10423.260009765625,
 10432.169921875,
 1

In [17]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=label_data.index, y=list(map(lambda x:x/init_investment, total)), name='Return Percentage'))

fig.update_layout(title="S&P500")
fig.show()
# daily_return = list(map(lambda idx_val: (idx_val[1] - total[idx_val[0] - 1]) / total[idx_val[0] - 1] if idx_val[0] > 0 else 0, enumerate(total)))
print('average_daily_return = ',((total[-1] / init_investment) - 1) / len(total))

average_daily_return =  0.00039695817102628834
